# Conjuntos de datos de entrenamiento, validación y prueba

**Refs**

https://www.geeksforgeeks.org/training-neural-networks-with-validation-using-pytorch/

https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import sklearn as skl
import pandas as pd
#from torchviz import make_dot
import torch.optim as optim
from collections import defaultdict
import pickle
import dill
import json
import datetime
try:
  import google.colab
  from google.colab import files  
  COLAB = True
except:
  COLAB = False

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Usando el dispositivo {}'.format(device))

Consideremos una familia de redes neuronales $y = f_a(x;w_a)$ de distintas arquitecturas $a$.
Aquí, $x$ denota la entrada (ej. features) de la red, $y$ la salida (ej. labels) y $w_a$ los parámetros o pesos sinápticos de la misma.

Redes de distintas arquitecturas pueden pueden aprender datasets de distintas estructuras.
En particular, redes de distintas arquitecturas pueden tener distintos números de parámetros y por ende pueden aprender datasets de distintas complejidades.

Sabemos que redes demasiado simples (con pocos parámetros) no logran aprender datasets suficientemente complejos, y que redes demasiado complejas tienden a sobrefitear datos.
Entonces, nos interesa encontrar aquella red de la familia, que mejor se desempeñe con los datos a disposición, aprendiendo correctamente los datos de entrenamiento, pero también generalizando sin sobrefitear sobre datos de validación.
Para ello, dividimos el conjunto de datos a disposición (el cuál se supone estar compuesto de muestras generadas de manera estadísticamente independiente) en tres conjuntos:

1. el conjunto de entrenamiento (training),

2. el conjunto de validación (validation), y

3. el conjunto de testeo (test).

Luego, para encontrar la red de arquitectura más conveniente, realizamos el siguiente procedimiento para cada arquitectura $a$:

1. Entrenamos la red $f_a(x,w_a)$ optimizando con respecto a $w_a$ sobre las muestras $x$ obtenidas de dataset de entrenamiento, y utilizando la función de pérdida de nuestra preferencia. Esto resulta en valores "optimos" de los parámetros $\hat{w}_a$, de manera que $f_a(x,\hat{w}_a)$ constituye la red entrenada.

2. Luego, usando métricas de nuestra preferencia (ej. la función de pérdida), evaluamos $f_a(x,\hat{w}_a)$ sobre el conjunto de validación, para ver cuán bien generaliza la red ya entrenada.

Luego, elegimos la arquitectura $\hat{a}$ que haya dado los mejores resultados durante el paso de validación 2.
Finalmente, caracterizamos las bondades de nuestra elección $f_{\hat{a}}(x;w_{\hat{a}})$ evaluándola sobre el conjunto de prueba (test).

Veamos un ejemplo con FashionMNIST y una red multicapa de sólo una capa oculta.
Para ello, comenzamos por crear los conjuntos de entrenamiento, validación y testeo.

In [ ]:
# La primera vez esto tarda un rato ya que tiene que bajar los datos de la red.
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}
train_dataset = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_dataset = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

Luego definimos la red neuronal.
Esta es un perceptron con una capa oculta de tamaño arbitrario $n$. 
En este ejemplo, dicho $n$ es el único grado de libertad que dejamos variar, de entre todos los que definen la arquitectura de la red.
En otras palabras, nuestra familia estará compuesta de redes con capas ocultas de distintos tamaños.

In [ ]:
class Net(nn.Module):
    def __init__(self,n=128):
        super(Net,self).__init__()
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.linear1 = nn.Linear(28*28,n)
        self.linear2 = nn.Linear(n,10)
    def forward(self,x):
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x

Implementamos las funciones para entrenar, validar y testear un modelo.

In [ ]:
# Definimos la función de entrenamiento
def train_loop(dataloader,model,loss_fn,optimizer,verbose_each=32):  
    # Calculamos cosas utiles que necesitamos
    num_samples = len(dataloader.dataset)
    # Seteamos el modelo en modo entrenamiento. Esto sirve para activar, por ejemplo, dropout, etc. durante la fase de entrenamiento.
    model.train()
    # Pasamos el modelo la GPU si está disponible.        
    model = model.to(device)    
    # Iteramos sobre lotes (batchs)
    for batch,(X,y) in enumerate(dataloader):
        # Pasamos los tensores a la GPU si está disponible.
        X = X.to(device)
        y = y.to(device)      
        # Calculamos la predicción del modelo y la correspondiente pérdida (error)
        pred = model(X)
        loss = loss_fn(pred,y)
        # Backpropagamos usando el optimizador proveido.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Imprimimos el progreso cada 100 batchs
        if batch % verbose_each*len(X) == 0:
            loss   = loss.item()
            sample = batch*len(X) # Número de batch * número de muestras en cada batch
            #print(f"batch={batch} loss={loss:>7f}  muestras-procesadas:[{sample:>5d}/{num_samples:>5d}]")            
# De manera similar, definimos la función de validación y testeo
def test_loop(dataloader,model,loss_fn):
    num_samples = len(dataloader.dataset)
    num_batches = len(dataloader)
    avrg_loss    = 0
    frac_correct = 0
    # Seteamos el modelo en modo evaluacion. Esto sirve para desactivar, por ejemplo, dropout, etc. cuando no estamos en una fase de entrenamiento.
    model.eval()
    # Pasamos el modelo la GPU si está disponible.    
    model = model.to(device)    
    # Para validar, desactivamos el cálculo de gradientes.
    with torch.no_grad():
        # Iteramos sobre lotes (batches)
        for X,y in dataloader:
            # Pasamos los tensores a la GPU si está disponible.
            X = X.to(device)
            y = y.to(device)           
            # Calculamos las predicciones del modelo...
            pred = model(X)
            # y las correspondientes pérdidas (errores), los cuales vamos acumulando en un valor total.
            avrg_loss += loss_fn(pred,y).item()
            # También calculamos el número de predicciones correctas, y lo acumulamos en un total.
            frac_correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    # Calculamos la pérdida total y la fracción de clasificaciones correctas, y las imprimimos.
    avrg_loss    /= num_batches
    frac_correct /= num_samples
    #print(f"Test Error: \n Accuracy: {frac_correct:>0.5f}, Avg. loss: {avrg_loss:>8f} \n")
    return avrg_loss,frac_correct

In [ ]:
# Definimos hiperparámetros de entrenamiento
learning_rate = 1e-3
batch_size = 500
num_epochs = 100
num_k = 1 #72
n=2048
# Creamos una funcion de perdida
loss_fn = nn.CrossEntropyLoss()
# Creamos un DataFrame de pandas para ir almacenando los valores calculados.
df = pd.DataFrame()
# Simulamos por tramos porque google colab se desconecta antes de que concluya para todos los valores de n en la lista.
# Recordar que 28*28=784
for k in range(num_k):
    # Creamos el modelo y el optimzador
    model = Net(n)
    # Creamos los dataloaders ...
    train_dataloader = DataLoader(train_dataset,batch_size=batch_size)
    # ... en particular, usamos el dataset de prueba (test) como dataset de validación
    valid_dataloader = DataLoader(test_dataset,batch_size=batch_size)         
    #optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
    optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate,eps=1e-08,weight_decay=0,amsgrad=False)
    # Entrenamos el modelo y calcualmos curvas.
    min_valid_loss = float("inf")
    for epoch in range(num_epochs):
        train_loop(train_dataloader,model,loss_fn,optimizer)
        train_loss,train_accu = test_loop(train_dataloader,model,loss_fn)
        valid_loss,valid_accu = test_loop(valid_dataloader,model,loss_fn)
        print(f"n={n} k={k} epoch={epoch} train_loss={train_loss} train_accu={train_accu} valid_loss={valid_loss} valid_accu={valid_accu}")
        df = df.append({"n":n,
                        "k":k,
                        "epoch":epoch,
                        "train_loss":train_loss,
                        "train_accu":train_accu,
                        "valid_loss":valid_loss,
                        "valid_accu":valid_accu}
                        ,ignore_index=True)
json_fname = "simulation-results-"+datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")+".json"
df.to_json(json_fname)
if COLAB:
    files.download(json_fname)

**Simulation summary**

simulation-results-2021-12-15-18-37-04.json

simulation-results-2021-12-15-18-59-55.json

simulation-results-2021-12-15-19-18-31.json

simulation-results-2021-12-15-19-42-48.json

simulation-results-2021-12-15-20-39-16.json

simulation-results-2021-12-15-21-10-38.json

simulation-results-2021-12-15-22-38-52.json

simulation-results-2021-12-15-23-46-54.json

In [ ]:
#df = pd.read_json(json_fname)
#df

In [ ]:
%%bash --out list_json
# Usamos el bash magic de Jupyter para ver que archivos *.json hemos creado.
# Guardamos el resultado en la variable list_json
ls *.json

In [ ]:
list_json = list_json.split()
list_json

In [ ]:
df = pd.concat([pd.read_json(json_fname) for json_fname in list_json],ignore_index=True)
df

In [ ]:
df1 = df.drop("k",1)
df1

In [ ]:
df2 = df1.pivot_table(index=["n","epoch"],aggfunc="count").reset_index()
df2

In [ ]:
df3 = df1.pivot_table(index=["n","epoch"],aggfunc="mean").reset_index()
df3

Visualicemos el desempeño de la red, graficando la loss y la accuracy vs el número de épocas de entrenamiento para los distintos tamaños de la capa oculta.

In [ ]:
fig,axes=plt.subplots(1,2)
fig.set_size_inches(10.0,5.0)
colors = cm.Dark2.colors
for color,n in zip(colors,df["n"].unique()):
    dfn = df3[df3["n"]==n]
    x = dfn["epoch"]
    ax = axes[0]
    ax.set_xlabel("epoch")
    ax.set_ylabel("loss")
    ax.plot(x,dfn["train_loss"],label=f"train n={n}",color=color)
    ax.plot(x,dfn["valid_loss"],label=f"valid n={n}",color=color,linestyle='--')
    ax.legend()
    ax = axes[1]
    ax.set_xlabel("epoch")
    ax.set_ylabel("accuracy")
    ax.plot(x,dfn["train_accu"],label=f"train n={n}",color=color)
    ax.plot(x,dfn["valid_accu"],label=f"test n={n}",color=color,linestyle='--')
    ax.legend()
fig.tight_layout()
plt.show()    

 Graficamos la min. loss y la max. accuracy en función del tamaño de la capa oculta. El min. loss y el max. accuracy se computan sobre las correspondientes curvas en función del número de épocas.

In [ ]:
df4 = df3.pivot_table(index=["n"],
                    aggfunc={
                        "train_loss":min,
                        "valid_loss":min,
                        "train_accu":max,
                        "valid_accu":max,
                    }
                   ).reset_index()
df4

In [ ]:
x=df4["n"]
fig,axes=plt.subplots(1,2)
fig.set_size_inches(10.0,5.0)
ax = axes[0]
ax.set_xlabel("n")
ax.set_ylabel("min. loss")
ax.scatter(x,df4["train_loss"],label=f"train")
ax.plot(x,df4["train_loss"],label=f"train")
ax.scatter(x,df4["valid_loss"],label=f"valid",linestyle='--')
ax.plot(x,df4["valid_loss"],label=f"valid",linestyle='--')
ax.set_xscale("log")
#ax.set_yscale("log")
ax.legend()
ax = axes[1]
ax.set_xlabel("n")
ax.set_ylabel("max. accuracy")
ax.scatter(x,df4["train_accu"],label=f"train")
ax.plot(x,df4["train_accu"],label=f"train")
ax.scatter(x,df4["valid_accu"],label=f"valid",linestyle='--')
ax.plot(x,df4["valid_accu"],label=f"valid",linestyle='--')
ax.set_xscale("log")
ax.legend()
fig.tight_layout()
plt.show()    